<a href="https://colab.research.google.com/github/SamuelRuby/DeepLearningProjects/blob/main/TRAIN_LLMS_ON_EMAILS_COPYFORGITHUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

In [ ]:
!pip install sumy

In [ ]:
!pip install wandb -qU



In [ ]:
#------------------#
# LIBRARIES
#-----------------------#

from google.colab import drive
import os
import re
import zipfile
import mailbox
from bs4 import BeautifulSoup  # Install if needed: !pip install beautifulsoup4
import csv
import json
import unicodedata
#import nltk #.tokenize import word_tokenize
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer

from torch.utils.data import Dataset

import wandb
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/Takeout')

In [ ]:
#unzipped
zip_file_path = 'takeout-20250401T135415Z-001.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()
print("Zip file extracted successfully!")

In [ ]:
file_list = os.listdir()
print(file_list)

In [ ]:
!cd Takeout

!ls

In [ ]:
os.listdir('Takeout/Mail')

In [ ]:
def extract_body(msg):
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == "text/plain":
                return part.get_payload(decode=True).decode('utf-8', errors='ignore')
            elif content_type == "text/html":
                soup = BeautifulSoup(part.get_payload(decode=True), "html.parser")
                return soup.get_text()
    else:
        return msg.get_payload(decode=True).decode('utf-8', errors='ignore')


In [ ]:
# Specify the path to the .mbox file:
mbox_file_path = '/content/drive/MyDrive/Takeout/Takeout/Mail/All mail Including Spam and Trash.mbox'

# mbox object:
mbox = mailbox.mbox(mbox_file_path)

# Iterate through messages:
for message in mbox:
  #print(message)
    # Access message headers and content, and labels
    subject = message['subject']
    sender = message['from']
    x_gmail_labels = message['X-Gmail-Labels']
    body = extract_body(message)#.get_payload()
    #delivered_to = message['Delivered-To']
    #received = message['Received']

    # print(f"Subject: {subject}")
    # print(f"Sender: {sender}")
    # print(f"X-Gmail-Labels: {x_gmail_labels}")
    # print(f"Body: {body[:200]}")
    # #print(f"Delivered-To: {delivered_to}")
    # #print(f"Received: {received}")
    # print("-" * 50)  # Separator


# X-Gmail-Labels: Archived,Important,Opened,Category personal,We could have
# Delivered-To: workwithrubyy@gmail.com
# Received: by
# Subject: Subject: Content-Type: Mime-Version: Date: Sender: Sender:
#  To: From: From: Subject: Subject: Content-Type: Mime-Version: Date:
#  Sender: Sender: X-Feedback-Id;
# X-Mailgun-Tag: CandidateAutoReply
# List-Unsubscribe:
# Message-Id:

In [ ]:
# save as csv
#output_file = "/content/drive/MyDrive/Takeout/emails.csv"

# with open(output_file, mode="w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Subject", "Sender", "Labels", "Body"])  # Column headers

#     for message in mbox:
#         subject = message["subject"]
#         sender = message["from"]
#         x_gmail_labels = message["X-Gmail-Labels"]
#         body = extract_body(message)

#         writer.writerow([subject, sender, x_gmail_labels, body])

# print(f"Emails saved to {output_file}")


In [ ]:
#save as json
emails = []

for message in mbox:
    email_data = {
        "subject": message["subject"],
        "sender": message["from"],
        "labels": message["X-Gmail-Labels"],
        "body": extract_body(message),
    }
    emails.append(email_data)

output_file = "/content/drive/MyDrive/Takeout/emails.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(emails, f, indent=4)

print(f"Emails saved to {output_file}")


In [ ]:
# # Filter emails with null or empty subject
# filtered_emails = [email for email in emails
#                    if email.get("subject") is None or email.get("subject") == ""]

# # Print the filtered emails (or do something else with them)
# for email in filtered_emails:
#     print(email)

2ND STEP: PREPROCESSING

In [ ]:
def clean_email_text(text):
  soup = BeautifulSoup(text, "html.parser") # Removes HTML tags
  text = soup.get_text()

  # Remove signatures and disclaimers using regex
  text = re.sub(r'(?i)(\n?--\n?.+|Thanks,?.*|Sincerely,?.*|Regards,?.*|Best,?.*|Unsubscribe.*|preferences.*)', '', text, flags=re.DOTALL)
  #text = re.sub(r'(?i)(Thanks,|Sincerely,|Regards,|Best,|Unsubscribe,|preferences,).*', '', text) # Common signature patterns
  text = re.sub(r'(?i)This email.*confidential.*', '', text, flags=re.DOTALL)  # Disclaimer example

  # Remove URLs using regex
  text = re.sub(r'http\S+', '', text)

  # Remove extra whitespace and newlines
  text = ' '.join(text.split())

  #   # Remove HTML tags:
  # if "<" in text and ">" in text:  # Check if HTML is likely present
  #     soup = BeautifulSoup(text, "html.parser")
  #     text = soup.get_text()

  # Remove non-ASCII characters:
  text = text.encode("ascii", "ignore").decode("ascii")

  # Normalize Unicode characters (e.g., fancy quotes, right-single quote, non-breaking spaces)
  text = unicodedata.normalize("NFKD", text)

  # Remove or replace emojis (using regex):
  text = re.sub(r'[\U0001F600-\U0001F64F]', '', text)  # removes all emojis
  # Or to remove them entirely:
  # text = re.sub(r'[\U0001F600-\U0001F64F]', '', text)

    # ... (Add more cleaning steps as needed) ...
  return text


# for email in emails:
#     email['body'] = clean_email_text(email['body'])

In [ ]:
#TOKENIZE AND VECTORIZATION
#NOTE: Ceell takes up to 15 mins to run, so don't run often

# nltk.download('punkt_tab')


# for email in emails:
#     email["cleaned_body"] = clean_email_text(email["body"])  # Clean the body text
#     email["tokens"] = word_tokenize(email["cleaned_body"])  # Tokenize

# # Save the updated data with cleaned text and tokens
# output_file = "/content/drive/MyDrive/Takeout/emails_cleaned_tokenized.json"
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(emails, f, indent=4)

# print(f"Cleaned emails saved to {output_file}")


model_name = "distilbert-base-uncased"


#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") #have to chnage to distilbert
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load pre-trained tokenizer and model:
model = AutoModel.from_pretrained(model_name)  # Use AutoModel for embedding generation


# CLEAN TEXT (using function) AND TOKENIZE
for email in emails:
    email["cleaned_body"] = clean_email_text(email["body"])  # Clean text

    # Tokenize using BERT's tokenizer
    email["encoded"] = tokenizer(
        email["cleaned_body"],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

#VECTORIZE
def tvectorize(tokenized_input):
  with torch.no_grad(): #saves memory cuz no gradient computation
    outputs = model(**tokenized_input)
  embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()  # Get [CLS] token embedding #converts the embedding from a PyTorch tensor to a NumPy array.
  return embeddings

# Apply:
# using BERT to convert the text of each email into a meaningful numerical vector representation (embedding).
for email in emails:
    email['embeddings'] = tvectorize(email['encoded'])

In [ ]:
email["cleaned_body"]

In [ ]:
#email["encoded"]

In [ ]:
# email['embeddings']

In [ ]:
# missing_subjects = [email for email in emails if email.get("subject") is None]
# print(f"Number of emails with missing subjects: {len(missing_subjects)}")

# for email in missing_subjects[:5]:  # Show first 5
#     print(email)



In [ ]:
#defining labels, # make sure to ignore system labels (inbox, starred...)

#mapping each email's labels to numbers
def assign_label(labels, label_mapping, email):
    """Assigns a label to an email based on existing labels or content-based logic."""

    # 1️⃣ Try using predefined labels first
    for label in labels.split(","):
        label = label.strip()  # Remove spaces or newline characters
        if label in label_mapping:
            return label_mapping[label]  # Return the first matching label

    # 2️⃣ Use subject/body-based logic if no label is found
    subject = (email.get("subject") or "").lower()
    body = (email.get("cleaned_body") or "").lower() # Use .get() with a default value


    if any(word in subject for word in ["regret", "we regret", "declined", "not moving forward"]) or \
       any(word in body for word in ["regret", "pursue other candidates", "to move forward", "not selected", "unsuccessful", "unfortunately"]):
        return label_mapping["We could have"]

    elif any(word in subject for word in ["interview", "assessment", "invitation", "schedule"]) or \
         any(word in body for word in ["interview", "assessment", "invitation", "schedule", "time slot", "HR meeting", "meeting scheduled"]):
        return label_mapping["Interviews"]

    elif "offer" in subject or "offer" in body:
        return label_mapping["Offer"]

    elif any(word in subject for word in ["order confirmation", "shipment", "your package", "envío", "order", "order status"]) or \
         any(word in body for word in ["receipt", "pedido", "tracking number", "order number", "tracking", "delivery", "purchase", "refund", "order", "order status", "delivery expected", "discount", "promo code", "order status", "return policy"]):
        return label_mapping["Shopping"]

    elif any(word in subject for word in ["invoice", "payment due", "billing statement", "tax", "refund", "salary", "bank statement"]) or \
         any(word in body for word in ["payment reminder", "amount due", "overdue invoice", "tax", "refund", "salary", "bank statement"]):
        return label_mapping["Finances"]

    elif any(word in subject for word in ["exam", "class schedule", "course enrollment"]) or \
         any(word in body for word in ["assignment", "university", "professor", "grades", "lecture", "syllabus", "research", "homework", "enrollment", "course", "semester", "assignment deadline"]):
        return label_mapping["School"]

    elif any(word in subject for word in ["application", "thank you", "your submission", "cover letter"]) or \
         any(word in body for word in ["thank you for your application", "application", "we received your application", "cover letter", "resume submitted", "position applied"]):
        return label_mapping["Applications"]

    elif any(word in subject for word in ["newsletter", "subscribe", "weekly update", "product update", "new feature", "community news"]) or \
         any(word in body for word in ["unsubscribe", "manage preferences", "stay informed", "product update", "new feature", "community news"]):
        return label_mapping["Updates"]

    return label_mapping.get("Other", -1)  # If no match is found, default to 'Other'

#define label mappings
label_mapping = {
    "Shopping": 0,
    "Updates": 1,
    "School": 2,
    "We could have": 3,
    "Socials": 4,
    "Forums": 5,
    "Promotions": 6,
    "Interviews": 7,
    "Offer": 8,
    "Finances": 9,
    "Applications": 10,
    "Other": 11
}

# Apply to each email:
for email in emails:
    email["numeric_label"] = assign_label(email["labels"], label_mapping, email)


**FINE TUNE A SMALL TRANSFORMER MODEL**
 - Choose model
 - train it on required tasks
 - evaluation and finetuning (tweaking)

MODEL REQUIREMENTS:
- One that's fast and efficient,
- capable of sequence classification,
- can be fine-tuned for email replies and few-shot learning, and
- is suitable for summarizing long email threads.

**choices**:

- DistilBERT: retains much of BERT's performance on various tasks, including sequence classification
BUT may not be the absoulte best for very long sequences or complicated tasks

- longformer: can handle much longer sequences than traditional transformers, making it potentially well-suited for long email threads and summarization tasks
BUT needs more computational resources


Due to our computational resources, we're going with DistilBERT


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))

In [ ]:
class EmailDataset(Dataset):
  def __init__(self, emails, tokenizer, label_mapping):
    self.emails = emails
    self.tokenizer = tokenizer
    self.label_mapping = label_mapping

  def __len__(self):
    return len(self.emails)

  def __getitem__(self, idx):
    email = self.emails[idx]
    text = email["cleaned_body"]
    label = email["numeric_label"]

    encoded_input = self.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    #return{**encoded_input, "labels": torch.tensor(label, dtype=torch.long)}
    return {"input_ids": encoded_input["input_ids"].squeeze(0), "attention_mask": encoded_input["attention_mask"].squeeze(0), "labels": torch.tensor(label, dtype=torch.long)}



In [ ]:
# Prepare dataset
dataset = EmailDataset(emails, tokenizer, label_mapping)  # Create the dataset

# Split into train and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=0.3, random_state=42)

# Training arguments
training_args = TrainingArguments(
    output_dir="./email_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5, #try 3e-5 (slightly higher) or 1e-5 (slightly lower).
    per_device_train_batch_size=8, #if run out of memory, do 8 instead
    per_device_eval_batch_size=16,
    num_train_epochs=5, #or 5. try 5 later
    weight_decay=0.01, #can increase to 0.05
    fp16=True,
    report_to="none",  # Disable wandb logging as I don't want to use it
    logging_dir='./logs',            # directory for storing logs
    logging_strategy='steps',       # log for every global_step
    logging_steps=10
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Train data
    eval_dataset=val_dataset  # Validation data
)

trainer.train()
#trainer.train(resume_from_checkpoint=True)


In [ ]:
trainer.evaluate()

In [ ]:
#evaluate model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer.compute_metrics = compute_metrics
trainer.evaluate()


In [ ]:
#test predictions
def classify_email(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    predicted_label = outputs.logits.argmax(-1).item()
    return predicted_label

new_email = "you just got admission into the university, congratulations!"
print(classify_email(new_email))  # See predicted label


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

predictions, labels, _ = trainer.predict(val_dataset)
preds = np.argmax(predictions, axis=1)
print(classification_report(labels, preds))


In [ ]:
label_mapping = {
    "Shopping": 0,
    "Updates": 1,
    "School": 2,
    "We could have": 3,
    "Socials": 4,
    "Forums": 5,
    "Promotions": 6,
    "Interviews": 7,
    "Offer": 8,
    "Finances": 9,
    "Applications": 10,
    "Other": 11
}

In [ ]:
Extractive Summarization vs. Abstractive
Now that  classification model is trained, summarization is a separate task.

- Extractive Summarization: Picks the most important sentences from the email.

- Abstractive Summarization: Generates a completely new summary.

can use extractive summarization because:
 - It's computationally cheaper.
 - More reliable for emails (e.g., extracting key points from long messages).

In [ ]:
email_text = """How's 2025 treating you - like a VIP\r\n(totally on point)? Or are you already feeling the January vibe after\r\na Detty December? (I'm sure you gerrit) \r\n\r\n_WELL, JANUARY MIGHT FEEL LIKE IT HAS 100 DAYS, BUT DON\u2019T WORRY,\r\nWITH ME, YOU WON\u2019T EVEN NOTICE THE CLOCK TICKING \ud83d\ude01\ud83d\ude01! _\r\n\r\nTo make it even spicier, this 2025, connect with your fellow stock\r\ninvestors, steal\u2026 sorry,\ud83e\udd23\ud83e\udd23 learn from their wins and losses,\r\nand drop your own market gist like the investment guru you are. \r\n\r\n_LET\u2019S MAKE THOSE NAIRA (AND DOLLARS) WORK HARDER THAN WE DO!\u00a0_ \r\n\r\nLet\u2019s dive right into the business of today!\r\n\r\n \t\t 20 Years?! Quantum Computing's Shocking Truth \r\n\r\nLast year, quantum computing stocks were a thing but this new year,\r\nsomeone But this year? Someone just slammed the brakes on the train,\r\nand let\u2019s just say\u2026_IT WASN\u2019T CUTE. \ud83d\ude2c_\r\n\r\nBefore I continue with the gist lemme tell you a little bit about the\r\nQuantum computing stocks. Basically, they are shares of companies\r\ndeveloping and investing in quantum computing technology. This\r\nemerging field uses quantum-mechanical phenomena to perform\r\ncalculations exponentially faster than classical computers. Companies\r\nlike Rigetti Computing ($RGTI), IonQ ($IONQ), and D-Wave Quantum\r\n($QBTS) are leading players in this space, attracting investors eager\r\nto capitalize on the potential breakthroughs.\r\n\r\n_BUT THEN THIS WEEK, JENSEN HUANG HAPPENED!!_ \r\n\r\nYep! Tech legend and Nvidia CEO Jensen Huang decided to casually pop\r\nthe bubble, telling analysts that \u201cvery useful quantum computers\u201d\r\nare still 20 years away. Yes, TWENTY. Not five. Not ten. Twenty\r\nyears!!!!. \ud83e\udee0 \r\n\r\nThe Street, of course, did not like the sound of that and of course\r\nyou know what follows next\u2026 _A DIP!!!_ Rigetti Computing saw its\r\nstock drop by 45%, IonQ fell off the cliff by 39%, D-Wave Quantum went\r\ndown by 36%, and Quantum Computing took a hit at 43%.\r\n\r\nWhat a heartbreak.\ud83d\udc94 \r\n\r\n(... rest of your email content ...)"""

# ... (your summarization code) ...

parser = PlaintextParser.from_string(email_text, Tokenizer("english")) # Pass email_text here
# ... (rest of your summarization code) ...

In [ ]:
a_text= """ Meta came with $800 million in hand, thinking it could do what it always does: buy a South Korean AI chip startup, absorb it, and move on. But guess what? The company straight-up said, “Nah, we’re good." Who rejects $800 million just like that?!

Whoa! FuriosaAI didn’t even flinch. Not a “let’s negotiate” or “make it $1 billion, and we’ll think about it.”

Uncle Mark got a cold, hard NO. I’m not saying I enjoy billionaire heartbreak, but this one? Took me OUT 🤣🤣. Because… when was the last time Uncle Mark didn’t get what he wanted? This man bought Instagram, snatched WhatsApp before anyone could blink and casually picked up Oculus. But FuriosaAI? Oh! They rejected his $800M. Ooops!

FuriosaAI is a small but mighty AI chip company, and Meta wanted them to boost its AI game. But these guys are playing the long game. They’d rather build their empire than become another Meta-owned brand. They’re not here to be part of Uncle Mark’s shopping cart. They want full control over their future.

Meanwhile, Uncle Mark is still on his AI buy-a-thon, pouring $65 billion into AI because he’s tired of depending on Nvidia. But this particular deal? It’s Dead on Arrival.

FuriosaAI is choosing the long road, betting on itself instead of taking an easy exit. Dollar Tree Just Sold a Whole Business for $1B… After Buying It for $9B
You know Dollar Tree, right? That big American discount store where everything is supposed to be cheap? Well, they just sold off their Family Dollar chain for $1 billion after buying it for $9 billion back in 2015.

Imagine buying something for 9,000 naira and selling it years later for 1,000 naira. Omo, this one is paining me on their behalf. A whole $8 billion loss. But funny enough, investors are actually CELEBRATING..Why? Because Family Dollar was that one friend who keeps borrowing money but never pays it back. (You know the type. If you don’t, you might BE the type. 👀)
Family Dollar wasn’t just struggling; it was fighting for its life against big players like Walmart, Amazon, Shein, and Temu.

Inflation also made things worse because people stopped spending on things like home decor and clothes, which are exactly the products Family Dollar needed to make real money.


Dollar Tree had been trying to offload this liability for nearly a year, and now they’ve finally found some private investors to take it off their hands.

The moment Dollar Tree cut off the dead weight, its stock price jumped 6% like a toxic ex finally leaving the group chat. But that’s not the end of their troubles... They still have wahala ahead. Yes!

With rising costs from global supply chain issues and new trade policies making imports more expensive, Dollar Tree is about to start bleeding an extra $20 million per month in costs, which means it's going to cost them $20 million per month to do business. 👀
Trump’s Tariff Saga Continues Another day, another tariff! This time, foreign automakers in the US are in the hot seat!

Yep, Trump just announced a 25% tariff on foreign-made cars and auto parts, and you know what? This is just the beginning because on April 2 (the new date for Liberation Day), more tariffs will hit imported goods from major U.S. trading partners.

And no, this isn’t some April Fool’s prank. Trump deliberately set the date for April 2 to avoid any confusion. Companies importing vehicles from Mexico and Canada under the USMCA (United States-Mexico-Canada Agreement) won’t immediately face the 25% tariff that Trump is imposing on foreign-made cars and auto parts.

Instead, they will receive "special consideration" (likely exemptions or delays) until the U.S. Commerce Department creates a formal process for applying these tariffs.

This move is part of Trump's strategy to push companies to manufacture more products within the U.S. instead of relying on imports. This announcement isn’t sitting so well with automakers across all nations, and some are saying this could backfire, making cars more expensive and leading to job losses instead of job creation.

What do you think?? I would like to hear your thoughts on Trove Social.
Waymo’s Next Stop is Washington, D.C. Waymo has been rolling out its robotaxis in different parts of the US, and now they’re bringing their robot taxis to Washington, D.C., America’s capital city.

But before you start dreaming of hopping into a car with no driver, calm down first.
 """

In [ ]:
text= """ Meta came with $800 million in hand, thinking it could do what it always does: buy a South Korean AI chip startup, absorb it, and move on. But guess
what? The company straight-up said, “Nah, we’re good." Who rejects $800 million just like that?!  Whoa! FuriosaAI didn’t even flinch. Not a “let’s negotiate” or
“make it $1 billion, and we’ll think about it.” Uncle Mark got a cold, hard NO. I’m not saying I enjoy billionaire heartbreak, but this one? Took me OUT.
Because… when was the last time Uncle Mark didn’t get what he wanted? This man bought Instagram, snatched WhatsApp before anyone could blink and casually picked
up Oculus. But FuriosaAI? Oh! They rejected his $800M. Ooops! FuriosaAI is a small but mighty AI chip company, and Meta wanted them to boost its AI game. But
these guys are playing the long game. They’d rather build their empire than become another Meta-owned brand. They’re not here to be part of Uncle Mark’s shopping
cart. They want full control over their future. Meanwhile, Uncle Mark is still on his AI buy-a-thon, pouring $65 billion into AI because he’s tired of
depending on Nvidia. But this particular deal? It’s Dead on Arrival. FuriosaAI is choosing the long road, betting on itself instead of taking an easy exit.
Dollar Tree Just Sold a Whole Business for $1B… After Buying It for $9B You know Dollar Tree, right? That big American discount store where everything is
supposed to be cheap? Well, they just sold off their Family Dollar chain for $1 billion after buying it for $9 billion back in 2015.

Imagine buying something for 9,000 naira and selling it years later for 1,000 naira. Omo, this one is paining me on their behalf. A whole $8 billion loss. But
funny enough, investors are actually CELEBRATING..Why? Because Family Dollar was that one friend who keeps borrowing money but never pays it back. (You know the
type. If you don’t, you might BE the type.) Family Dollar wasn’t just struggling; it was fighting for its life against big players like Walmart, Amazon, Shein,
and Temu.

Inflation also made things worse because people stopped spending on things like home decor and clothes, which are exactly the products Family Dollar needed to
make real money. Dollar Tree had been trying to offload this liability for nearly a year, and now they’ve finally found some private investors to take it off
their hands.  The moment Dollar Tree cut off the dead weight, its stock price jumped 6% like a toxic ex finally leaving the group chat. But that’s not the end
of their troubles... They still have wahala ahead. Yes! With rising costs from global supply chain issues and new trade policies making imports more expensive,
Dollar Tree is about to start bleeding an extra $20 million per month in costs, which means it's going to cost them $20 million per month to do business.
Trump’s Tariff Saga Continues Another day, another tariff! This time, foreign automakers in the US are in the hot seat!  Yep, Trump just announced a 25% tariff
on foreign-made cars and auto parts, and you know what? This is just the beginning because on April 2 (the new date for Liberation Day), more tariffs will hit
imported goods from major U.S. trading partners. And no, this isn’t some April Fool’s prank. Trump deliberately set the date for April 2 to avoid any confusion.
Companies importing vehicles from Mexico and Canada under the USMCA (United States-Mexico-Canada Agreement) won’t immediately face the 25% tariff that Trump is
imposing on foreign-made cars and auto parts. Instead, they will receive "special consideration" (likely exemptions or delays) until the U.S. Commerce
Department creates a formal process for applying these tariffs. This move is part of Trump's strategy to push companies to manufacture more products within the
U.S. instead of relying on imports. This announcement isn’t sitting so well with automakers across all nations, and some are saying this could backfire, making
cars more expensive and leading to job losses instead of job creation. What do you think?? I would like to hear your thoughts on Trove Social.
Waymo’s Next Stop is Washington, D.C. Waymo has been rolling out its robotaxis in different parts of the US, and now they’re bringing their robot taxis to
Washington, D.C., America’s capital city. But before you start dreaming of hopping into a car with no driver, calm down first. For now, the taxis will still
have a safety driver sitting behind the wheel, just in case something goes wrong. The government has rules that don’t allow fully driverless taxis yet, so
they’re still in the testing phase, mapping the roads and learning the city's layout.  If all goes well, by next year, people in Washington will be able to order
a driverless taxi through the Waymo app; no driver to speak with, no steering, just pure tech magic. Waymo is already operating in Phoenix, Los Angeles, San
Francisco, and Austin, where they even partnered with Uber. Now, they’re expanding to Miami and Washington next year, and they also plan to launch in Atlanta
later this year.

 """

In [ ]:
#extractive summarization
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

parser = PlaintextParser.from_string(text, Tokenizer("english"))
summarizer = LsaSummarizer()
summary = summarizer(parser.document, 3)  # Extract top 3 sentences

for sent in summary:
    print(sent)
#print(" ".join(str(sent) for sent in summary))


In [ ]:
SUMMARIZATION WORKS 60% WELL

CLASSIFICATION-- NOT SO GOOD

In [ ]:
#abstractive summarization

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5-small (lightweight version for Colab)
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def summarize_email(email_text, max_length=50):
    # Prepend "summarize: " for T5
    input_text = "summarize: " + email_text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Generate summary
    summary_ids = model.generate(inputs.input_ids, max_length=max_length, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:

# Example email
email_text = """
The error LookupError: Resource punkt_tab not found. indicates that the necessary NLTK data package punkt_tab is not downloaded. This package provides tokenizers (tools that break text into sentences and words) needed by the sumy library for text summarization.

The sumy library relies on NLTK's punkt tokenizer to segment the text into sentences. If punkt isn't available, it can't process the text correctly. The code attempts to load this tokenizer, but since the data isn't present, it throws a LookupError.
"""
print(summarize_email(email_text))


OMPLEMENTING FEW SHOT LEARNING FOR EMAIL CLASSIFICATION

Few-shot learning allows your model to quickly adapt to new email categories with minimal labeled examples. 3 main categories:

- Fine-Tune with Additional Data (Standard Approach). If you get a new category (e.g., "Medical Appointments"), fine-tune the model again with a few labeled examples

- Use Prompt-Based Learning (For Transformers like GPT, T5). Instead of retraining, some models like T5 and GPT-based models can learn new categories dynamically from a prompt.

- Use Embeddings + Nearest Neighbor for New Classes. Instead of retraining, store embeddings of past emails and classify new emails by comparing similarity.

In [ ]:
#embeddings use case
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")  # Small & fast

# Existing category embeddings
category_texts = {
    "Urgent": "Immediate action needed!",
    "Job": "Interview scheduled.",
    "Shopping": "Your order is confirmed."
}
category_embeddings = {cat: model.encode(text) for cat, text in category_texts.items()}

# New email
new_email = "order should arrive by Friday."
new_email_embedding = model.encode(new_email)

# Find the closest category
similarities = {cat: np.dot(new_email_embedding, emb) for cat, emb in category_embeddings.items()}
predicted_category = max(similarities, key=similarities.get)
print(predicted_category)


In [ ]:
#prompt based learning use case
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = "t5-small"
t5_model = T5ForConditionalGeneration.from_pretrained(model_name) # Renamed to avoid conflict

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = SentenceTransformer("all-MiniLM-L6-v2")
input_text = "Classify this email: 'Your appointment is on Friday.' \n Categories: Urgent, Job, Shopping, Medical"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = t5_model.generate(inputs.input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
